In [2]:
import numpy as np
import matplotlib.pyplot
from matplotlib.pyplot import imshow
import scipy.special as scipy
from PIL import Image
import codecs, json 

class NeuralNetwork(object):
    def __init__(self):
        self.input_neuron_count = 28*28  # One for each pixel, 28*28 = 784 in total.
        self.hidden_neuron_count =200 # Arbitraty.
        self.output_neuron_count = 11 # One for each digit from ፩ to ፻.
        self.learning_rate = 0.1 # Arbitraty.

       
       
        
        generate_random_weight_matrix = lambda input_neuron_count, output_neuron_count: ( 
            np.random.normal(0.0,  pow(input_neuron_count, -0.5), (input_neuron_count, output_neuron_count))
        )

        self.input_x_hidden_weights = generate_random_weight_matrix(self.input_neuron_count, self.hidden_neuron_count)
        self.hidden_x_output_weights = generate_random_weight_matrix(self.hidden_neuron_count, self.output_neuron_count)

        self.ai = np.ones((self.input_neuron_count, 1))
        self.ah = np.ones((self.hidden_neuron_count, 1))
        self.ao = np.ones((self.output_neuron_count, 1))
        
#         self.changew_input = np.zeros((self.num_hidden, self.num_input))
#         self.changew_output = np.zeros((self.num_output, self.num_hidden))

#         self.changeb_output = np.zeros((self.num_output, 1))
#         self.changeb_hidden = np.zeros((self.num_hidden, 1))

        self.activation_function = lambda value: scipy.expit(value) # Sigmoid function
    def dsigmoid(self, a):
        return a * (1 - a)
        
    def feed_forward(self, inputs):
        inputs = np.insert(inputs, 0, 1)
        inputs = inputs.reshape(self.input_neuron_count ,1)
        self.ai[:] = inputs

        sumi = np.dot(self.wi, self.ai)
        self.ah[:] = self.activate(sumi)
        self.ah[0] = 1

        sumh = np.dot(self.wo, self.ah)
        self.ao = self.activate(sumh, 'output')

        return self.ao[:]
            
    def back_propagate(self, target, learning_rate):
        delta_output = np.ones((self.output_neuron_count, 1))
        delta_hidden = np.ones((self.hidden_neuron_count, 1))

#         self.dsigmoid = np.vectorize(self.dsigmoid, otypes=[np.float])
#         self.dReLU = np.vectorize(self.dReLU, otypes=[np.float])

        cost = - (target - self.ao)

        delta_output = cost * self.dsigmoid(self.ao)
        

#         update weights and biases

#         change = np.dot(delta_output, self.ah.transpose())
        self.wo += learning_rate * change + self.changew_output
#         self.changew_output = change

#         change = np.dot(delta_hidden, self.ai.transpose())
        self.wi += learning_rate * change + self.changew_input
#         self.changew_input = change

        loss = 0.5 * (cost ** 2)
        return loss.sum()


    def fit(self, inputs, targets, epoch=100, learning_rate=0.0002):
        for i in range(epoch):
            loss = 0.0
            print("iteration ", i+1)
            for j in range(len(inputs)):
                input = inputs[j]
                self.feed_forward(input)

                loss += self.back_propagate(targets[j], learning_rate)
            print("loss , " , loss / len(inputs))
    def train(self, input_array, target_array):
         #convert inputs list into 2d array
        inputs = np.array(input_array, ndmin=2)
        targets = np.array(target_array, ndmin=2)
#         inputs = self.back_propagate(inputs,0.0002)
        
       #calculate the signals into the hidden layer
        hidden_layer_input = np.dot(inputs, self.input_x_hidden_weights)
          #calculate the signals emerging from the hidden layer
        hidden_layer_output = self.activation_function(hidden_layer_input)

        output_layer_input = np.dot(hidden_layer_output, self.hidden_x_output_weights)
        output_layer_output = self.activation_function(output_layer_input)

        output_errors = targets - output_layer_output
        self.hidden_x_output_weights += self.learning_rate * np.dot(hidden_layer_output.T, (output_errors * output_layer_output * (1 - output_layer_output)))

        hidden_errors = np.dot(output_errors, self.hidden_x_output_weights.T)
        self.input_x_hidden_weights += self.learning_rate * np.dot(inputs.T, (hidden_errors * hidden_layer_output * (1 - hidden_layer_output)))

    def query(self, input_array):
        inputs = np.array(input_array, ndmin=2)

        hidden_layer_input = np.dot(inputs, self.input_x_hidden_weights)
        hidden_layer_output = self.activation_function(hidden_layer_input)

        output_layer_input = np.dot(hidden_layer_output, self.hidden_x_output_weights)
        output_layer_output = self.activation_function(output_layer_input)

        return output_layer_output
    
    def export(self):
        input_x_hidden_weights = self.input_x_hidden_weights.tolist()
        json.dump(input_x_hidden_weights, codecs.open('input_x_hidden_weights.json', 'w', encoding='utf-8'), separators=(',', ':'), sort_keys=True, indent=4)
        hidden_x_output_weights = self.hidden_x_output_weights.tolist()
        json.dump(hidden_x_output_weights, codecs.open('hidden_x_output_weights.json', 'w', encoding='utf-8'), separators=(',', ':'), sort_keys=True, indent=4)
    
def prepare_data(handwritten_digit_array):
    return ((handwritten_digit_array / 255.0 * 0.99) + 0.0001).flatten()
    
def create_target(digit_target):
    target = np.zeros(11) + 0.01
    target[digit_target] = target[digit_target] + 0.98
    return target

def get_index_of_max(array):
    array = array[0]
    index = 0
    m = max(array)
    for n in array:
        if n == m:
            return index
        index = index + 1

# Training
neural_network = NeuralNetwork()
training_data_file = open('geez_train.csv', 'r')
training_data = training_data_file.readlines()
training_data_file.close()

# Training / First epoch
for data in training_data:
    handwritten_digit_raw = data.split(',')
    handwritten_digit_array = np.asfarray(handwritten_digit_raw[1:]).reshape((28, 28))
    handwritten_digit_target = int(handwritten_digit_raw[0])
    neural_network.train(prepare_data(handwritten_digit_array), create_target(handwritten_digit_target))

# Training / Second epoch
for data in training_data:
    handwritten_digit_raw = data.split(',')
    handwritten_digit_array = np.asfarray(handwritten_digit_raw[1:]).reshape((28, 28))
    handwritten_digit_target = int(handwritten_digit_raw[0])
    neural_network.train(prepare_data(handwritten_digit_array), create_target(handwritten_digit_target))
    
# Testing
test_data_file = open('geez_test.csv', 'r')
test_data = test_data_file.readlines()
test_data_file.close()

test_count = 0
hit_count = 0
miss_count =0
for data in test_data:
    #np.set_printoptions(precision=4, threshold=10000)
    handwritten_digit_raw = data.split(',')
    handwritten_digit_array = np.asfarray(handwritten_digit_raw[1:]).reshape((28, 28))
    handwritten_digit_target = int(handwritten_digit_raw[0])
    output = neural_network.query(handwritten_digit_array.flatten())
    
    hit_or_miss = get_index_of_max(output)
    if hit_or_miss == handwritten_digit_target:
        hit_count = hit_count + 1
    else:
        miss_count = miss_count + 1
    test_count = test_count + 1
    
print("test_count", test_count)
print("hit_count", hit_count)
print("miss_count", miss_count)
print("hit ratio", hit_count/test_count * 100)

neural_network.export();



test_count 100
hit_count 31
miss_count 69
hit ratio 31.0


NameError: name 'np' is not defined

In [ ]:
x

In [1]:
# Previe of test image and result of recognition
%matplotlib inline  
test_image_index = 7
test_data_file = open('mnist_test.csv', 'r')
test_data = test_data_file.readlines()
test_data_file.close()
data = test_data[test_image_index]
handwritten_digit_raw = data.split(',')
handwritten_digit_array = np.asfarray(handwritten_digit_raw[1:]).reshape((28, 28))
handwritten_digit_target = int(handwritten_digit_raw[0])
matplotlib.pyplot.imshow(handwritten_digit_array, cmap='Greys', interpolation='None')
output = neural_network.query(handwritten_digit_array.flatten())
print(output)
print(get_index_of_max(output))

FileNotFoundError: [Errno 2] No such file or directory: 'mnist_test.csv'